# Transformer (tylko decoder), inny encoding. 5 featerow - mag, x, y, depth, diff_days

In [24]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [25]:
ds_train = tf.data.Dataset.load("../data/ds_train/")
ds_val = tf.data.Dataset.load("../data/ds_val/")
ds_test = tf.data.Dataset.load("../data/ds_test/")

In [26]:
BATCH_SIZE = 1024
BLOCK_SIZE = 32

In [27]:
def transform_element(features, labels):
    # transform lebels from (1,) to (2,)
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features[:,:-1], features[:,-1]), labels[0]

In [28]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [29]:
for x, y in ds_train.take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(y.shape)

(32, 4)
(32,)
(2,)


In [30]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [31]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.lstm = tf.keras.layers.LSTM(d_model // 5 * 4, return_sequences=True)
        self.embed_dd = tf.keras.layers.Embedding(100, d_model // 5)
        self.pos_encoding = tf.keras.layers.Embedding(100, d_model)
        self.conc = tf.keras.layers.Concatenate()

    def call(self, x):
        cont, dd = x
        x1 = self.lstm(cont)
        x2 = self.embed_dd(dd)
        x = self.conc([x1, x2])
        x_pos = self.pos_encoding(tf.range(x.shape[1]))
        x = x + x_pos
        return x

In [32]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [33]:
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [34]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [35]:
class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self,
                d_model,
                num_heads,
                dff,
                dropout_rate=0.1):
        super(DecoderBlock, self).__init__()

        self.causal_self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.causal_self_attention(x=x)
        x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
        return x

In [36]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_blocks = [
            DecoderBlock(d_model=d_model, num_heads=num_heads,
                        dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

    def call(self, x):
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.dropout(x)
        for block in self.dec_blocks:
            x = block(x)
        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [37]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads,
                dropout_rate=0.1):
        super().__init__()
        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(2)

    def call(self, inputs):
        x = self.decoder(inputs) # (batch_size, target_len, d_model)
        x = x[:, -1, :]
        logits = self.final_layer(x)  # (batch_size, 1, target_vocab_size)
        return logits

In [38]:
model = Transformer(num_layers=2, d_model=65, num_heads=4)

In [39]:
model.build(input_shape=[(None, 32, 4), (None, 32)])
model.count_params()

225448

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [20]:
history = model.fit(ds_train, epochs=20, validation_data=ds_val)

Epoch 1/20


2023-11-05 05:28:20.009604: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 172 of 2000
2023-11-05 05:28:39.961593: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 506 of 2000
2023-11-05 05:28:49.989928: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 665 of 2000
2023-11-05 05:29:00.004370: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 841 of 2000
2023-11-05 05:29:19.976043: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 1212 of 2000
2023-11-05 05:29:30.005378: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a

3190/3190 [==============================] - 1026s 282ms/step - loss: 0.2529 - accuracy: 0.9086 - f1_score: 0.6645 - val_loss: 0.2251 - val_accuracy: 0.9225 - val_f1_score: 0.7046
Epoch 2/20


2023-11-05 05:45:19.823707: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 174 of 2000
2023-11-05 05:45:39.786228: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 520 of 2000
2023-11-05 05:45:49.808644: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 694 of 2000
2023-11-05 05:45:59.813070: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 867 of 2000
2023-11-05 05:46:19.827426: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 1214 of 2000
2023-11-05 05:46:29.827945: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a

3190/3190 [==============================] - 1247s 355ms/step - loss: 0.2175 - accuracy: 0.9161 - f1_score: 0.7158 - val_loss: 0.2158 - val_accuracy: 0.9240 - val_f1_score: 0.6973
Epoch 3/20


...

3190/3190 [==============================] - 1242s 354ms/step - loss: 0.1949 - accuracy: 0.9251 - f1_score: 0.7433 - val_loss: 0.2190 - val_accuracy: 0.9208 - val_f1_score: 0.7101
Epoch 19/20


2023-11-05 11:37:25.379009: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 178 of 2000
2023-11-05 11:37:35.422921: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 356 of 2000
2023-11-05 11:37:55.385641: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 707 of 2000
2023-11-05 11:38:05.391095: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 885 of 2000
2023-11-05 11:38:25.411679: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 1241 of 2000
2023-11-05 11:38:45.426656: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a

3190/3190 [==============================] - 1243s 354ms/step - loss: 0.1957 - accuracy: 0.9242 - f1_score: 0.7404 - val_loss: 0.2020 - val_accuracy: 0.9254 - val_f1_score: 0.6268
Epoch 20/20


2023-11-05 11:58:08.593531: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 180 of 2000
2023-11-05 11:58:28.547974: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 536 of 2000
2023-11-05 11:58:38.593498: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 714 of 2000
2023-11-05 11:58:48.595308: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 892 of 2000
2023-11-05 11:58:58.608231: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 1070 of 2000
2023-11-05 11:59:18.562306: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a

3190/3190 [==============================] - 1242s 354ms/step - loss: 0.1947 - accuracy: 0.9248 - f1_score: 0.7395 - val_loss: 0.1972 - val_accuracy: 0.9267 - val_f1_score: 0.6931


In [21]:
model.evaluate(ds_test)

100/100 [==============================] - 14s 137ms/step - loss: 0.1342 - accuracy: 0.9571 - f1_score: 0.7696


[0.13415437936782837,
 0.9570908546447754,
 array([0.9774408 , 0.56184524], dtype=float32)]

In [22]:
model.save("../models/model_v4.keras")

In [23]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v4.csv", index=False)